In [36]:
# Importamos las librerías
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import openpyxl
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from scipy import sparse

In [3]:
# Ruta del PDF
df1 = pd.read_excel('/Users/fernandopradagorge/Documents/Documentos - MacBook Pro de Fernando/Universidad/Master Data Science and AI/Nuclio Projects/Projects/Evolución ventas vehículos particulares/Lista detallada de ventas.xlsx')

In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7907 entries, 0 to 7906
Data columns (total 36 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   V                      7906 non-null   object        
 1   Fecha                  7904 non-null   datetime64[ns]
 2   V/A                    7904 non-null   object        
 3   IDV                    7904 non-null   float64       
 4   Origen                 7904 non-null   object        
 5   Referencia             7904 non-null   float64       
 6   Tipo Vehículo          7686 non-null   object        
 7   Matricula              7814 non-null   object        
 8   Marca                  7904 non-null   object        
 9   Modelo                 7904 non-null   object        
 10  Canal                  7904 non-null   object        
 11  Facturación            7905 non-null   float64       
 12  Facturación (media)    7905 non-null   float64       
 13  Imp

In [5]:
df1.head()

,V,Fecha,V/A,IDV,Origen,Referencia,Tipo Vehículo,Matricula,Marca,Modelo,...,Cambio,Kilómetros,Fecha Matric.,Segmento,SubSegmento,Importe Remarketing,Margen I,Margen II,Margen III,Margen IV
0,1,2024-01-18,V,156385.0,VO,31114932.0,MOTOS,6457GSW,PIAGGIO,PIAGGIO VESPA 50CC,...,AUTOMATICO,72940.0,2010-01-18,NaN,NaN,0.00,82.65,82.65,-48.56,-48.56
1,1,2023-03-21,V,156385.0,VO,28637837.0,MOTOS,6457GSW,PIAGGIO,PIAGGIO VESPA 50CC,...,AUTOMATICO,72940.0,2010-01-18,NaN,NaN,0.00,413.22,413.22,476.84,476.84
2,1,2023-06-29,V,159474.0,VO,29507497.0,TURISMOS,5383GTM,FORD,FOCUS,...,MANUAL,124380.0,2010-02-22,Turismos,Segmento C,0.00,991.73,991.73,855.35,855.35
3,1,2024-03-18,V,200955.0,VO,31567110.0,TURISMOS,4134HSR,OPEL,OPEL CORSA,...,MANUAL,106772.0,2013-08-29,Turismos,Segmento B,1347.61,4123.97,4123.97,2861.92,2861.92
4,1,2023-01-16,V,205707.0,VO,28058110.0,TURISMOS,1962HTZ,ALFA,GIULIETTA,...,MANUAL,66511.0,2013-12-18,Turismos,Segmento C,0.00,2300.51,2300.51,1618.87,1618.87


### EDA

In [6]:
df1.columns

Index(['V', 'Fecha', 'V/A', 'IDV', 'Origen', 'Referencia', 'Tipo Vehículo',
       'Matricula', 'Marca', 'Modelo', 'Canal', 'Facturación',
       'Facturación (media)', 'Importe Venta (media)', 'Margen', 'Financiado',
       'Imp. Financiado', 'Interés', 'Financiera', 'NumeroCuotas',
       'Concesionario', 'Vendedor', 'Comprador', 'Días en Stock', 'Bastidor',
       'Combustible', 'Cambio', 'Kilómetros', 'Fecha Matric.', 'Segmento',
       'SubSegmento', 'Importe Remarketing', 'Margen I', 'Margen II',
       'Margen III', 'Margen IV'],
      dtype='object')

In [7]:
df1 = df1.drop(['Referencia', 'Canal', 'Matricula', 'Financiado', 'Imp. Financiado', 'Interés', 'Financiera', 'NumeroCuotas', 'Vendedor', 'Comprador', 'Bastidor', 'Kilómetros', 'V', 'V/A', 'IDV', 'Margen I', 'Margen II', 'Margen III', 'Margen IV', 'Segmento', 'SubSegmento', 'Fecha Matric.', 'Cambio', 'Días en Stock', 'Importe Remarketing', 'Margen'], axis = 1)

In [8]:
df1.columns

Index(['Fecha', 'Origen', 'Tipo Vehículo', 'Marca', 'Modelo', 'Facturación',
       'Facturación (media)', 'Importe Venta (media)', 'Concesionario',
       'Combustible'],
      dtype='object')

In [9]:
df2 = df1.copy()

In [10]:
df2.columns

Index(['Fecha', 'Origen', 'Tipo Vehículo', 'Marca', 'Modelo', 'Facturación',
       'Facturación (media)', 'Importe Venta (media)', 'Concesionario',
       'Combustible'],
      dtype='object')

In [11]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7907 entries, 0 to 7906
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Fecha                  7904 non-null   datetime64[ns]
 1   Origen                 7904 non-null   object        
 2   Tipo Vehículo          7686 non-null   object        
 3   Marca                  7904 non-null   object        
 4   Modelo                 7904 non-null   object        
 5   Facturación            7905 non-null   float64       
 6   Facturación (media)    7905 non-null   float64       
 7   Importe Venta (media)  7905 non-null   float64       
 8   Concesionario          7904 non-null   object        
 9   Combustible            7224 non-null   object        
dtypes: datetime64[ns](1), float64(3), object(6)
memory usage: 617.9+ KB


In [12]:
df2 = df2.drop(['Concesionario', 'Tipo Vehículo'], axis = 1)

In [13]:
df2.isnull().sum()

Fecha                      3
Origen                     3
Marca                      3
Modelo                     3
Facturación                2
Facturación (media)        2
Importe Venta (media)      2
Combustible              683
dtype: int64

In [14]:
# Eliminar los 3 nulos de la columna 'Fecha' ya que no son significativos para el modelo
df2.dropna(subset = ['Fecha'], inplace = True)

In [15]:
# Establecer la columan 'Fecha' como índice 
df2.set_index('Fecha', inplace = True)

### Imputación de valores nulos

In [17]:
# Definir las transformaciones para las columnas numéricas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy = 'median')),
    ('scaler', MinMaxScaler())
])

In [18]:
# Definir las transfformaciones para las columnas categórica
categorical_transformer = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy = 'most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown ='ignore'))
])

In [20]:
# Crear el transformador de columnas usando el ColumnTransform
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, df2.select_dtypes(include=['float64', 'int64']).columns),
    ('cat', categorical_transformer, df2.select_dtypes(include=['object']).columns)
])

In [21]:
# Crear la pipeline
pipeline = Pipeline(steps = [
    ('preprocessor', preprocessor)
])

In [22]:
# Aplicar el pipeline al DataFrame
df2_transformed = pipeline.fit_transform(df2)

In [29]:
print("Forma del DataFrame transformado:", df2_transformed.shape)

Forma del DataFrame transformado: (7904, 431)


In [37]:
print("Verificación final antes de crear el DataFrame:")
print("Forma del df2_transformed:", df2_transformed.shape)
print("Dimensiones esperadas de new_columns:", len(new_columns))

Verificación final antes de crear el DataFrame:
Forma del df2_transformed: (7904, 431)
Dimensiones esperadas de new_columns: 431


In [38]:
# Convertir la matriz dispersa a una matriz densa si es necesario
if sparse.issparse(df2_transformed):
    print("Converting sparse matrix to dense matrix...")
    df2_transformed = df2_transformed.toarray()

if df2_transformed.shape[1] == len(new_columns):
    df2_final = pd.DataFrame(df2_transformed, index=df2.index, columns=new_columns)
    print("DataFrame final creado con éxito:")
    print(df2_final.head())
else:
    print("Incongruencia en el número de columnas:", df2_transformed.shape[1], "esperado:", len(new_columns))


Converting sparse matrix to dense matrix...
DataFrame final creado con éxito:
            Facturación  Facturación (media)  Importe Venta (media)  x0_KM0  \
Fecha                                                                         
2024-01-18     0.380065             0.009954               0.008965     0.0   
2023-03-21     0.378384             0.007269               0.006546     0.0   
2023-06-29     0.386024             0.019470               0.018843     0.0   
2024-03-18     0.409839             0.057503               0.051786     0.0   
2023-01-16     0.424827             0.081440               0.073342     0.0   

            x0_VN  x0_VO  x1_ABARTH  x1_ALFA  x1_APRILIA  x1_AUDI  ...  \
Fecha                                                              ...   
2024-01-18    0.0    1.0        0.0      0.0         0.0      0.0  ...   
2023-03-21    0.0    1.0        0.0      0.0         0.0      0.0  ...   
2023-06-29    0.0    1.0        0.0      0.0         0.0      0.0  ...  

In [39]:
df2_final.head()

,Facturación,Facturación (media),Importe Venta (media),x0_KM0,x0_VN,x0_VO,x1_ABARTH,x1_ALFA,x1_APRILIA,x1_AUDI,...,x2_ZONTES,x2_ZS,x3_1.0 Ecoboost 100cv,x3_DIESEL,x3_ELECTRICO,x3_GASOLINA,x3_HIBRIDO,x3_HIBRIDO ENCHUFABLE,x3_MICROHIBRIDO,x3_MICROHIBRIDO DIESEL
Fecha,,,,,,,,,,,,,,,,,,,,,
2024-01-18,0.380065,0.009954,0.008965,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2023-03-21,0.378384,0.007269,0.006546,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2023-06-29,0.386024,0.019470,0.018843,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2024-03-18,0.409839,0.057503,0.051786,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-01-16,0.424827,0.081440,0.073342,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
